In [61]:
import numpy as np

from implementations import *
from proj1_helpers import *

In [41]:
# Load data
DATA_TRAIN_PATH = 'train.csv' 
y, tX, ids, headers = load_csv_data(DATA_TRAIN_PATH) # Modified the load_csv_data to also give headers

In [11]:
headers.shape

(30,)

In [12]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [58]:
data[0].shape

(250000, 54)

In [57]:
data, targets, ids = process_features_train(tX, headers, y, 3)

In [52]:
 w_1 = logistic_regression_demo(targets[0], data[0], max_iters=500, gamma=0.01)

Current iteration=0, training loss=[nan]
Current iteration=499, training loss=[nan]
Proportion test error:  [nan]


In [30]:
data[0].shape

(250000, 54)

In [31]:
targets[0].shape

(250000,)

In [36]:
def final_model(tX, headers, y, degree, tX_test, ids_test, headers_test):
    # process features for to train the model
    data, targets, ids = process_features_train(tX, headers, y, degree)
    
    # train base model 
    w_1 = logistic_regression_demo(targets[0], data[0], max_iters=500, gamma=0.01)

    # train jet=1 model using base model weights as initial weights
    w_2 = logistic_regression_demo_winit(targets[1], data[2], max_iters=500, gamma=0.01)

    # train jet=2/3 model using base model weights as initial weights
    w_3 = logistic_regression_demo_winit(targets[2], data[4], max_iters=500, gamma=0.01)
    
    

    # process test set
    data, ids = process_features_test(tX_test, headers, ids_test, degree)

    # create Predictions
    weights = [w_1, w_2, w_3]
    y_pred_final = create_predictions(weights, data, ids)
    
    return y_pred_final

In [37]:
def cross_validation(y, x, k_indices, k, gamma, degree):
    """return the loss of ridge regression."""
    # ***************************************************
    # get k'th subgroup in test, others in train
    x_tr = x[k_indices[np.arange(len(k_indices))!=k].ravel()]
    x_te = x[k_indices[k]]
    
    y_tr = y[k_indices[np.arange(len(k_indices))!=k].ravel()]
    y_te = y[k_indices[k]]
    
    ids_tr = ids[k_indices[np.arange(len(k_indices))!=k].ravel()]
    ids_te = ids[k_indices[k]]
    
    # ***************************************************
    # form data with polynomial degree
    print("Fold number: {f}, polynomial degree: {d}".format(f=k+1, d=degree))

    # ***************************************************
    max_iters=100
    # INSERT THE MODEL
    y_pred_te = final_model(x_tr, headers, y_tr, degree, x_te, ids_te, headers)
    
    # ***************************************************
    # Calcualte the accuracy
    acc = np.sum(y_pred_te==y_te)/len(y_te)
    
    return acc


In [38]:
def cross_validation_demo(y, x, degrees):
    seed = 1
    k_fold = 5
    
    max_iter = 10000
    threshold = 1e-8
    gamma = 0.01
    
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    accuracy = np.zeros((degrees, k_fold))
    
    
    for degree in np.arange(degrees)+1:
        acc_d = []
        for k in range(k_fold):
            acc = cross_validation(y, x, k_indices, k, gamma, degree)
            acc_d.append(acc)
        accuracy[degree-1] = np.array(acc_d)
    
    return accuracy

In [39]:
degrees = 1
cross_validation_demo(y, tX, degrees)

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
# process features for to train the model
degree = 3
data, targets, ids = process_features_train(tX, headers, y, degree)


# train base model 
w_1 = logistic_regression_demo(targets[0], data[0], max_iters=50000, gamma=0.01)

# train jet=1 model using base model weights as initial weights
w_2 = logistic_regression_demo_winit(targets[1], data[2], max_iters=15000, gamma=0.01)

# train jet=2/3 model using base model weights as initial weights
w_3 = logistic_regression_demo_winit(targets[2], data[4], max_iters=15000, gamma=0.01)